In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM,Dense,Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.dates as mdates
from sklearn import linear_model

In [ ]:
df_final = pd.read_csv("../input/stock-price-edatime-series-analysis/individual_stocks_5yr/GOOGL_data.csv",na_values=['null'],index_col='date',parse_dates=True,infer_datetime_format=True)
df_final.head()

In [ ]:
df_final.drop(df_final.columns[[5]], axis=1, inplace=True)
df_final.head()

In [ ]:
df_final.shape

# **Plotting the close column**

In [ ]:
df_final['close'].plot()

# **Correlating the features with close**

In [ ]:
D = df_final.drop(['close'],axis=1)

In [ ]:
D.corrwith(df_final['close']).plot.bar(
        figsize = (20, 10), title = "Correlation with Close Feature", fontsize = 20,
        rot = 90, grid = True)

In [ ]:
#We can see that all other features except volume are positively correlated
test = df_final
# Target column
target = pd.DataFrame(test['close'])
display(test.head())

In [ ]:
feature_columns = ['open', 'high', 'low', 'volume']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
feature_minmax_transform_data = scaler.fit_transform(test[feature_columns])
feature_minmax_transform = pd.DataFrame(columns=feature_columns, data=feature_minmax_transform_data, index=test.index)
feature_minmax_transform.head()

In [ ]:
print('Shape of features : ', feature_minmax_transform.shape)
print('Shape of target : ', target.shape)

In [ ]:
target = target.shift(-1)
validation_y = target[-100:-1]
target = target[:-100]

In [ ]:
validation_X = feature_minmax_transform[-100:-1]
feature_minmax_transform = feature_minmax_transform[:-100]
display(validation_X.tail())
display(validation_y.tail())

In [ ]:
print('Shape of features : ', feature_minmax_transform.shape)
print('Shape of target : ', target.shape)
display(target.tail())

# **Splitting into training and testing sets**

In [ ]:
ts_split= TimeSeriesSplit(n_splits=10)
for train_index, test_index in ts_split.split(feature_minmax_transform):
        X_train, X_test = feature_minmax_transform[:len(train_index)], feature_minmax_transform[len(train_index): (len(train_index)+len(test_index))]
        y_train, y_test = target[:len(train_index)].values.ravel(), target[len(train_index): (len(train_index)+len(test_index))].values.ravel()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

# **Processing the data**

In [ ]:
X_train =np.array(X_train)
X_test =np.array(X_test)

X_tr_t = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_tst_t = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# **Model making**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM
K.clear_session()
model_lstm = Sequential()
model_lstm.add(LSTM(16, input_shape=(1, X_train.shape[1]), activation='relu', return_sequences=False))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
history_model_lstm = model_lstm.fit(X_tr_t, y_train, epochs=200, batch_size=8, verbose=1, shuffle=False, callbacks=[early_stop])

# **Evaluation of lstm model**

In [ ]:
y_pred_test_lstm = model_lstm.predict(X_tst_t)
y_train_pred_lstm = model_lstm.predict(X_tr_t)
print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(y_train, y_train_pred_lstm)))
r2_train = r2_score(y_train, y_train_pred_lstm)

print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test, y_pred_test_lstm)))
r2_test = r2_score(y_test, y_pred_test_lstm)

In [ ]:
score_lstm= model_lstm.evaluate(X_tst_t, y_test, batch_size=1)

In [ ]:
print('LSTM: %f'%score_lstm)

In [ ]:
y_pred_test_LSTM = model_lstm.predict(X_tst_t)

# **Plotting the predictions**

In [ ]:
plt.plot(y_test, label='True')
plt.plot(y_pred_test_LSTM, label='LSTM')
plt.title("LSTM's_Prediction")
plt.xlabel('Observation')
plt.ylabel('INR_Scaled')
plt.legend()
plt.show()

In [ ]:
from sklearn import metrics
print('Avg. error in predicting score =',metrics.mean_absolute_error(y_test,y_pred_test_lstm))